In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [3]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1330k  100 1330k    0     0  1437k      0 --:--:-- --:--:-- --:--:-- 1436k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1253k  100 1253k    0     0  3544k      0 --:--:-- --:--:-- --:--:-- 3551k


In [4]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d85de947-0031-4bb8-8070-0637974f75a1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=04c87ef17bf2423928bcb44e9ecfb1def62c72f81d4c0e476ec4b7465f36e1fd




Accessing as Pepe-Chuy

Initialized MLflow to track repo "Pepe-Chuy/nyc-taxi-time-prediction"

Repository Pepe-Chuy/nyc-taxi-time-prediction initialized!

https://dagshub.com/Pepe-Chuy/nyc-taxi-time-prediction.mlflow


2024/09/17 21:38:14 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/f89c177353e742bf83c2fd3dab8421bb', creation_time=1726630694466, experiment_id='0', last_update_time=1726630694466, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [10]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [11]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("Pepe-Chuy/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [12]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [14]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [15]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:7.08037                           
[1]	validation-rmse:6.08695                           
[2]	validation-rmse:5.68642                           
[3]	validation-rmse:5.45905                           
[4]	validation-rmse:5.39898                           
[5]	validation-rmse:5.36173                           
[6]	validation-rmse:5.33633                           
[7]	validation-rmse:5.34047                           
[8]	validation-rmse:5.33504                           
[9]	validation-rmse:5.33277                           
[10]	validation-rmse:5.32857                          
[11]	validation-rmse:5.32411                          
[12]	validation-rmse:5.32355                          
[13]	validation-rmse:5.32204                          
[14]	validation-rmse:5.31880                          
[15]	validation-rmse:5.31715                          
[16]	validation-rmse:5.31483                          
[17]	validation-rmse:5.31485                          
[18]	valid

2024/09/17 21:39:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pepechuy/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available."

2024/09/17 21:40:12 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:40:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pepechuy/.local/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:40:12] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:40:16 WAR

[0]	validation-rmse:8.44908                                                    
[1]	validation-rmse:7.89313                                                    
[2]	validation-rmse:7.43106                                                    
[3]	validation-rmse:7.04830                                                    
[4]	validation-rmse:6.72078                                                    
[5]	validation-rmse:6.45996                                                    
[6]	validation-rmse:6.24774                                                    
[7]	validation-rmse:6.06237                                                    
[8]	validation-rmse:5.92025                                                    
[9]	validation-rmse:5.80621                                                    
[10]	validation-rmse:5.70759                                                   
[11]	validation-rmse:5.62946                                                   
[12]	validation-rmse:5.56917            

2024/09/17 21:41:06 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:41:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pepechuy/.local/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:41:06] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:41:08 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/17 21:41:08 WARNING mlflow.models.model: M

[0]	validation-rmse:8.76598                                                     
[1]	validation-rmse:8.44788                                                     
[2]	validation-rmse:8.15502                                                     
[3]	validation-rmse:7.88593                                                     
[4]	validation-rmse:7.63878                                                     
[5]	validation-rmse:7.41247                                                     
[6]	validation-rmse:7.20596                                                     
[7]	validation-rmse:7.01726                                                     
[8]	validation-rmse:6.84500                                                     
[9]	validation-rmse:6.68782                                                     
[10]	validation-rmse:6.54508                                                    
[11]	validation-rmse:6.41578                                                    
[12]	validation-rmse:6.29802

...

2024/09/17 21:46:06 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:46:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pepechuy/.local/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:06] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:46:09 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/17 21:46:09 WARNING mlflow.models.model: M

[0]	validation-rmse:8.63518                                                    
[1]	validation-rmse:8.21613                                                    
[2]	validation-rmse:7.84959                                                    
[3]	validation-rmse:7.52810                                                    
[4]	validation-rmse:7.24963                                                    
[5]	validation-rmse:7.00734                                                    
[6]	validation-rmse:6.79832                                                    
[7]	validation-rmse:6.61661                                                    
[8]	validation-rmse:6.46018                                                    
[9]	validation-rmse:6.32669                                                    
[10]	validation-rmse:6.20961                                                   
[11]	validation-rmse:6.11069                                                   
[12]	validation-rmse:6.02628            

2024/09/17 21:46:28 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:46:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pepechuy/.local/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:28] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:46:31 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2024/09/17 21:46:31 WARNING mlflow.models.model: M

100%|██████████| 10/10 [07:03<00:00, 42.40s/trial, best loss: 5.165399741667276]
[0]	validation-rmse:6.70058
[1]	validation-rmse:5.78346
[2]	validation-rmse:5.45757
[3]	validation-rmse:5.33358
[4]	validation-rmse:5.28587
[5]	validation-rmse:5.26172
[6]	validation-rmse:5.24846
[7]	validation-rmse:5.24440
[8]	validation-rmse:5.24324
[9]	validation-rmse:5.24207
[10]	validation-rmse:5.24138
[11]	validation-rmse:5.23581
[12]	validation-rmse:5.23184
[13]	validation-rmse:5.22892
[14]	validation-rmse:5.22822
[15]	validation-rmse:5.22481
[16]	validation-rmse:5.22255
[17]	validation-rmse:5.21967
[18]	validation-rmse:5.21972
[19]	validation-rmse:5.21828
[20]	validation-rmse:5.21784
[21]	validation-rmse:5.21697
[22]	validation-rmse:5.21605
[23]	validation-rmse:5.21480
[24]	validation-rmse:5.21482
[25]	validation-rmse:5.21170
[26]	validation-rmse:5.20960
[27]	validation-rmse:5.20714
[28]	validation-rmse:5.20518
[29]	validation-rmse:5.20428
[30]	validation-rmse:5.20227
[31]	validation-rmse:5.19930
[

2024/09/17 21:46:59 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/17 21:46:59 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pepechuy/.local/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:46:59] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/17 21:47:01 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/09/17 21:47:01 WARNING mlflow.models.model: Mode

In [16]:
#1f3021e29d92414abcd95df10e108a5e
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Successfully registered model 'nyc-taxi-model'.
2024/09/17 21:49:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


In [17]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726631353590, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-17 21:49:15.828433', last_updated_timestamp=1726631356058, name='nyc-taxi-model', run_id='1f3021e29d92414abcd95df10e108a5e', run_link='', source='mlflow-artifacts:/f89c177353e742bf83c2fd3dab8421bb/1f3021e29d92414abcd95df10e108a5e/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [18]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([19.032124, 28.269821,  9.230973, ..., 42.209858, 13.918596,
       19.760328], dtype=float32)